# Using neural classification
As has been proven by [Wang (2017)](https://arxiv.org/abs/1705.00648), neural classifiers carry better results than non-neural classifiers when detecting fake news. However, it is unknown how well neural networks classify fake news when using previously mentioned text embeddings. 
In this notebook, the second research question will be answered: *how well do neural network architecture classify fake news compared to non-neural classification algorithms?*

<hr>

## On the usage of neural networks
Literature on CNNs and Bi-LSTMs


In [1]:
# General imports
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import keras
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, Reshape, Conv1D, Flatten

# Set offline mode for plotly
init_notebook_mode(connected = True)

# The DataLoader class gives access to pretrained vectors from the Liar dataset
from data_loader import DataLoader
data = DataLoader()

Using TensorFlow backend.


In [2]:
general = data.get_dfs()

# Recode labels from 6 to 3
def recode(label):
    if label == 'false' or label == 'pants-fire' or label == 'barely-true':
        return 0
    elif label == 'true' or label == 'mostly-true':
        return 2
    elif label == 'half-true':
        return 1

for dataset in general.keys():
    general[dataset]['label'] = general[dataset]['label'].apply(lambda label: recode(label))

<hr>

## Bidirectional LSTMs

In [3]:
bert = data.get_bert()

In [3]:
# Get max-pooled BERT embeddings from RQ1
def max_pool(statement):
    if len(statement) > 1:
        return [row.max() for row in np.transpose([[token_row.max() for token_row in np.transpose(np.array(sentence))] for sentence in statement])]
    else:
        return [token_row.max() for token_row in np.transpose(statement[0])]

max_pooled_bert = {
    dataset: pd.DataFrame(list(bert[dataset].statement.apply(lambda statement: max_pool(statement)).values))
    for dataset in bert.keys()
}

In [6]:
def get_bilstm_score(X_train, X_test, X_validation, y_train = general['train']['label'], y_test = general['test']['label'], y_validation = general['validation']['label'], reshape = True):
    # Rearrange data types    
    params = locals().copy()    
    inputs = {
        dataset: np.array(params[dataset])
        for dataset in params.keys()
    }
    
    for dataset in inputs.keys():
        if dataset[0:1] == 'X' and reshape:
            # Reshape datasets from 2D to 3D
            inputs[dataset] = np.reshape(inputs[dataset], (inputs[dataset].shape[0], inputs[dataset].shape[1], 1))
        elif dataset[0:1] == 'y':
            inputs[dataset] = np_utils.to_categorical(np.array(inputs[dataset]), 3)
    
    # Set model parameters
    epochs = 5
    batch_size = 32
    input_shape = X_train.shape

    # Create the model
    model = Sequential()
    model.add(Bidirectional(LSTM(64, input_shape = input_shape)))
    model.add(Dropout(0.8))
    model.add(Dense(3, activation = 'softmax'))
    model.compile('sgd', 'categorical_crossentropy', metrics = ['accuracy']) 
    
    # Fit the training set over the model and correct on the validation set
    model.fit(inputs['X_train'], inputs['y_train'],
            batch_size = batch_size,
            epochs = epochs,
            validation_data = (inputs['X_validation'], inputs['y_validation']))
    
    # Get score over the test set
    score, acc = model.evaluate(inputs['X_test'], inputs['y_test'])
    
    return acc

In [5]:
get_bilstm_score(max_pooled_bert['train'], max_pooled_bert['test'], max_pooled_bert['validation'])

Instructions for updating:
Colocations handled automatically by placer.
2019-05-21 12:40:37,047 From /Users/martijn/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
2019-05-21 12:40:37,448 From /Users/martijn/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
2019-05-21 12:40:37,537 From /Users/martijn/anaconda3/lib/python3.6/site-packages/tensorflo

0.43715415052745654

Apparently, the condensed datasets from RQ1 do not perform well when using a neural classifier. The next step is trying out a padding approach.

In [65]:
%%time

# Store accuracies
accuracies = {
    padding_len: 0.0 for padding_len in list(range(5,36))
}

concatenated_bert = {
    dataset: [np.concatenate(np.array(statement)) for statement in bert[dataset].statement]
    for dataset in bert.keys()
}

for max_len in accuracies.keys():
    padded_bert = {
        dataset: sequence.pad_sequences(concatenated_bert[dataset], maxlen = max_len, dtype = float)
        for dataset in concatenated_bert.keys()
    }
    
    accuracies[max_len] = get_bilstm_score(padded_bert['train'], padded_bert['test'], padded_bert['validation'], reshape = False)

Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 69s 7ms/step - loss: 1.1184 - acc: 0.4145 - val_loss: 1.0170 - val_acc: 0.5093
Epoch 2/5
10235/10235 [==============================] - 17s 2ms/step - loss: 1.0597 - acc: 0.4532 - val_loss: 1.0196 - val_acc: 0.5031
Epoch 3/5
10235/10235 [==============================] - 16s 2ms/step - loss: 1.0432 - acc: 0.4707 - val_loss: 1.0158 - val_acc: 0.5125
Epoch 4/5
10235/10235 [==============================] - 16s 2ms/step - loss: 1.0362 - acc: 0.4775 - val_loss: 1.0134 - val_acc: 0.5117
Epoch 5/5
1265/1265 [==============================] - 2s 2ms/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 46s 4ms/step - loss: 1.1094 - acc: 0.4181 - val_loss: 1.0184 - val_acc: 0.4907
Epoch 2/5
10235/10235 [==============================] - 18s 2ms/step - loss: 1.0558 - acc: 0.4556 - val_loss: 1.0125 - val_acc: 0.5125
Epoch 3/5
10235/102

10235/10235 [==============================] - 31s 3ms/step - loss: 1.0350 - acc: 0.4852 - val_loss: 1.0104 - val_acc: 0.5101
Epoch 4/5
10235/10235 [==============================] - 30s 3ms/step - loss: 1.0312 - acc: 0.4890 - val_loss: 1.0051 - val_acc: 0.5140
Epoch 5/5
1265/1265 [==============================] - 3s 3ms/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 91s 9ms/step - loss: 1.1168 - acc: 0.4210 - val_loss: 1.0127 - val_acc: 0.5047
Epoch 2/5
10235/10235 [==============================] - 37s 4ms/step - loss: 1.0508 - acc: 0.4668 - val_loss: 1.0166 - val_acc: 0.5086
Epoch 3/5
10235/10235 [==============================] - 33s 3ms/step - loss: 1.0364 - acc: 0.4793 - val_loss: 1.0032 - val_acc: 0.5171
Epoch 4/5
10235/10235 [==============================] - 32s 3ms/step - loss: 1.0281 - acc: 0.4893 - val_loss: 1.0000 - val_acc: 0.5179
Epoch 5/5
1265/1265 [==============================] - 4s 3ms/step
Train on 10

In [67]:
bert_rounds

[{5: 0.513043478637816,
  6: 0.5122529648038239,
  7: 0.5075098817998712,
  8: 0.5083003953747127,
  9: 0.49407114631573673,
  10: 0.5098814233018476,
  11: 0.507509881752753,
  12: 0.5035573126299108,
  13: 0.4996047434599503,
  14: 0.5027667987488004,
  15: 0.507509881752753,
  16: 0.5090909091144682,
  17: 0.5067193679187609,
  18: 0.5114624509227135,
  19: 0.5098814233018476,
  20: 0.5162055336203971,
  21: 0.5193675893097527,
  22: 0.5193675893097527,
  23: 0.5233201584797131,
  24: 0.5217391304583417,
  25: 0.5106719371358397,
  26: 0.5106719367824524,
  27: 0.5177865616417685,
  28: 0.5075098817998712,
  29: 0.513043478637816,
  30: 0.5043478264639029,
  31: 0.5162055336203971,
  32: 0.5075098817998712,
  33: 0.5114624509698318,
  34: 0.513833992471808,
  35: 0.5075098817998712},
 {5: 0.5098814233018476,
  6: 0.5098814229484603,
  7: 0.49723320160458684,
  8: 0.5090909091144682,
  9: 0.5051383399445077,
  10: 0.505928854131887,
  11: 0.498814229272571,
  12: 0.5114624509698318,


In [69]:
traces = [round1, round2, round3, round4, round5]

# Create traces
bert_trace = go.Scatter(
    x = list(round1.keys()),
    y = list(round1.values()),
    mode = 'lines+markers',
    name = 'BERT'
)

def create_scatter(counter):
    acc_dict = traces[counter]
    
    return go.Scatter(
        x = list(acc_dict.keys()),
        y = list(acc_dict.values()),
        mode = 'lines+markers',
        name = 'Round ' + str(counter)
    )

trace_data = [create_scatter(trace) for trace in range(len(traces))]

layout = go.Layout(
    title = 'Test set accuracy of padded BERT dataset with variable maximum lengths',
)

fig = go.Figure(data = trace_data, layout = layout)

iplot(fig)

In [3]:
elmo = data.get_elmo()

In [4]:
def calculate_round(dataset):
    # Store accuracies
    accuracies = {
        padding_len: 0.0 for padding_len in list(range(5,36))
    }

    for max_len in accuracies.keys():
        padded_dataset = {
            fold: sequence.pad_sequences(dataset[fold], maxlen = max_len, dtype = float)
            for fold in dataset.keys()
        }

        accuracies[max_len] = get_bilstm_score(padded_dataset['train'], padded_dataset['test'], padded_dataset['validation'], reshape = False)

    return accuracies

In [7]:
%%time

concatenated_elmo = {
    fold: [np.concatenate(np.array(statement)) for statement in elmo[fold]['statement']]
    for fold in elmo.keys()
}

elmo_rounds = [calculate_round(concatenated_elmo) for round in range(5)]

Instructions for updating:
Colocations handled automatically by placer.
2019-06-04 08:49:28,085 From /Users/martijn/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
2019-06-04 08:49:29,218 From /Users/martijn/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
2019-06-04 08:49:29,336 From /Users/martijn/anaconda3/lib/python3.6/site-packages/tensorflo

10235/10235 [==============================] - 34s 3ms/step - loss: 1.1207 - acc: 0.4320 - val_loss: 1.0211 - val_acc: 0.5016
Epoch 2/5
10235/10235 [==============================] - 31s 3ms/step - loss: 1.0517 - acc: 0.4702 - val_loss: 1.0160 - val_acc: 0.5101
Epoch 3/5
10235/10235 [==============================] - 31s 3ms/step - loss: 1.0308 - acc: 0.4884 - val_loss: 1.0031 - val_acc: 0.5101
Epoch 4/5
10235/10235 [==============================] - 31s 3ms/step - loss: 1.0219 - acc: 0.4962 - val_loss: 1.0021 - val_acc: 0.5132
Epoch 5/5
1265/1265 [==============================] - 1s 1ms/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 38s 4ms/step - loss: 1.1081 - acc: 0.4299 - val_loss: 1.0038 - val_acc: 0.5148
Epoch 2/5
10235/10235 [==============================] - 34s 3ms/step - loss: 1.0480 - acc: 0.4731 - val_loss: 1.0029 - val_acc: 0.5140
Epoch 3/5
10235/10235 [==============================] - 35s 3ms/step - loss: 

1265/1265 [==============================] - 8s 7ms/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 112s 11ms/step - loss: 1.0812 - acc: 0.4300 - val_loss: 1.0226 - val_acc: 0.5125
Epoch 2/5
10235/10235 [==============================] - 89s 9ms/step - loss: 1.0550 - acc: 0.4587 - val_loss: 1.0114 - val_acc: 0.5132
Epoch 3/5
10235/10235 [==============================] - 73s 7ms/step - loss: 1.0377 - acc: 0.4820 - val_loss: 1.0068 - val_acc: 0.5109
Epoch 4/5
10235/10235 [==============================] - 74s 7ms/step - loss: 1.0326 - acc: 0.4815 - val_loss: 1.0002 - val_acc: 0.5210
Epoch 5/5
1265/1265 [==============================] - 10s 8ms/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 114s 11ms/step - loss: 1.0779 - acc: 0.4358 - val_loss: 1.0203 - val_acc: 0.5125
Epoch 2/5
10235/10235 [==============================] - 94s 9ms/step - loss: 1.0488 - acc: 0

In [15]:
elmo_rounds

[{5: 0.5177865616417685,
  6: 0.5241106722665869,
  7: 0.5114624509698318,
  8: 0.5067193676124919,
  9: 0.5067193679187609,
  10: 0.5059288540847687,
  11: 0.49802371574484783,
  12: 0.5011857710808162,
  13: 0.49169960507291105,
  14: 0.49723320191085574,
  15: 0.4932806327408953,
  16: 0.5169960478077764,
  17: 0.5154150200455556,
  18: 0.513043478637816,
  19: 0.5083003955396268,
  20: 0.5185770754286423,
  21: 0.5067193679187609,
  22: 0.5146245063058001,
  23: 0.5011857710808162,
  24: 0.5106719370887214,
  25: 0.5169960477606581,
  26: 0.5193675893097527,
  27: 0.5114624509227135,
  28: 0.5130434785906977,
  29: 0.5241106723137052,
  30: 0.5169960477606581,
  31: 0.5169960477606581,
  32: 0.5146245063058001,
  33: 0.5185770754286423,
  34: 0.5051383402507766,
  35: 0.5035573125827925},
 {5: 0.5122529647567056,
  6: 0.5169960478077764,
  7: 0.5146245062586818,
  8: 0.5233201584325948,
  9: 0.5138339924246897,
  10: 0.4996047434599503,
  11: 0.49090909123891896,
  12: 0.4909090912

In [14]:
traces = elmo_rounds

# Create traces
def create_scatter(counter):
    acc_dict = traces[counter]
    
    return go.Scatter(
        x = list(acc_dict.keys()),
        y = list(acc_dict.values()),
        mode = 'lines+markers',
        name = 'Round ' + str(counter)
    )

trace_data = [create_scatter(trace) for trace in range(len(traces))]

layout = go.Layout(
    title = 'Test set accuracy of padded ELMo dataset with variable maximum lengths',
)

fig = go.Figure(data = trace_data, layout = layout)

iplot(fig)

In [14]:
x = list(range(5, 36))
x_rev = x[::-1]

# BERT
bert_matrix = np.transpose(np.array([np.array(list(acc_round.values())) for acc_round in bert_rounds]))
bert_y = [np.average(row) for row in bert_matrix]
bert_y_upper = [row.max() for row in bert_matrix]
bert_y_lower = [row.min() for row in bert_matrix]
bert_y_lower = bert_y_lower[::-1]

bert1 = go.Scatter(
    x = x + x_rev,
    y = bert_y_upper + bert_y_lower,
    fill = 'tozerox',
    fillcolor = 'rgba(0,100,80,0.2)',
    line = dict(color = 'rgba(255,255,255,0)'),
    showlegend = False,
    name = 'BERT',
)
bert2 = go.Scatter(
    x = x,
    y = bert_y,
    line = dict(color='rgb(0,100,80)'),
    mode = 'lines+markers',
    name = 'BERT',
)

# ELMo
elmo_matrix = np.transpose(np.array([np.array(list(acc_round.values())) for acc_round in elmo_rounds]))
elmo_y = [np.average(row) for row in elmo_matrix]
elmo_y_upper = [row.max() for row in elmo_matrix]
elmo_y_lower = [row.min() for row in elmo_matrix]
elmo_y_lower = elmo_y_lower[::-1]

elmo1 = go.Scatter(
    x = x + x_rev,
    y = elmo_y_upper + elmo_y_lower,
    fill = 'tozerox',
    fillcolor = 'rgba(0,176,246,0.2)',
    line = dict(color = 'rgba(255,255,255,0)'),
    showlegend = False,
    name = 'ELMo',
)
elmo2 = go.Scatter(
    x = x,
    y = elmo_y,
    line = dict(color='rgb(0,176,246)'),
    mode = 'lines+markers',
    name = 'ELMo',
)


data = [bert1, bert2, elmo1, elmo2]
layout = go.Layout(
    title = 'Bi-LSTM test set accuracy of padded datasets with variable maximum lengths',
    paper_bgcolor = 'rgb(255,255,255)',
    plot_bgcolor = 'rgb(229,229,229)',
    xaxis = dict(
        gridcolor = 'rgb(255,255,255)',
        range = [5,35],
        showgrid = True,
        showline = False,
        showticklabels = True,
        tickcolor = 'rgb(127,127,127)',
        ticks = 'outside',
        zeroline = False
    ),
    yaxis=dict(
        gridcolor='rgb(255,255,255)',
        showgrid = True,
        showline = False,
        showticklabels = True,
        tickcolor = 'rgb(127,127,127)',
        ticks = 'outside',
        zeroline = False
    ),
)

fig = go.Figure(data = data, layout = layout)

iplot(fig)

<hr>

## Convolutional neural networks

In [4]:
def get_cnn_score(X_train, X_test, X_validation, y_train = general['train']['label'], y_test = general['test']['label'], y_validation = general['validation']['label'], reshape = True):
    # Rearrange data types    
    params = locals().copy()    
    inputs = {
        dataset: np.array(params[dataset])
        for dataset in params.keys()
    }
    
    # Reshape datasets
    for dataset in inputs.keys():
        if dataset[0:1] == 'X':
            if reshape:
                inputs[dataset] = np.reshape(inputs[dataset], (inputs[dataset].shape[0], inputs[dataset].shape[1], 1))
            
        elif dataset[0:1] == 'y':
            inputs[dataset] = np_utils.to_categorical(np.array(inputs[dataset]), 3)
            
    # Set model parameters
    epochs = 5
    batch_size = 32
    input_shape =  inputs['X_train'].shape
    
    # Create the model
    model = Sequential()
    model.add(Conv1D(128, kernel_size = 2, activation='relu', input_shape = (input_shape[1], input_shape[2]), data_format = 'channels_first'))
    model.add(Conv1D(128, kernel_size = 3, activation='relu'))
    model.add(Conv1D(128, kernel_size = 4, activation='relu'))
    model.add(Dropout(0.8))
    model.add(Flatten())
    model.add(Dense(3, activation = 'softmax'))
    model.compile('sgd', 'categorical_crossentropy', metrics = ['accuracy']) 
    
    # Fit the training set over the model and correct on the validation set
    model.fit(inputs['X_train'], inputs['y_train'],
            batch_size = batch_size,
            epochs = epochs,
            validation_data = (inputs['X_validation'], inputs['y_validation']))
    
    # Get score over the test set
    score, acc = model.evaluate(inputs['X_test'], inputs['y_test'])
    print(acc)
    return acc

In [5]:
concatenated_bert = {
    dataset: [np.concatenate(np.array(statement)) for statement in bert[dataset].statement]
    for dataset in bert.keys()
}

In [6]:
def calculate_round(dataset):
    # Store accuracies
    accuracies = {
        padding_len: 0.0 for padding_len in list(range(5,36))
    }

    for max_len in accuracies.keys():
        padded_dataset = {
            fold: sequence.pad_sequences(dataset[fold], maxlen = max_len, dtype = float)
            for fold in dataset.keys()
        }

        accuracies[max_len] = get_cnn_score(padded_dataset['train'], padded_dataset['test'], padded_dataset['validation'], reshape = False)
        
    print(accuracies)
    return accuracies

In [10]:
%%time

cnn_bert_rounds = [calculate_round(concatenated_bert) for round in range(5)]

Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 116s 11ms/step - loss: 1.0628 - acc: 0.4374 - val_loss: 1.0260 - val_acc: 0.4938
Epoch 2/5
10235/10235 [==============================] - 115s 11ms/step - loss: 1.0405 - acc: 0.4690 - val_loss: 1.0127 - val_acc: 0.5078
Epoch 3/5
10235/10235 [==============================] - 112s 11ms/step - loss: 1.0328 - acc: 0.4813 - val_loss: 1.0137 - val_acc: 0.5117
Epoch 4/5
10235/10235 [==============================] - 114s 11ms/step - loss: 1.0260 - acc: 0.4858 - val_loss: 1.0025 - val_acc: 0.5156
Epoch 5/5
1265/1265 [==============================] - 5s 4ms/step
0.5067193679658791
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 112s 11ms/step - loss: 1.0658 - acc: 0.4321 - val_loss: 1.0336 - val_acc: 0.4766
Epoch 2/5
10235/10235 [==============================] - 115s 11ms/step - loss: 1.0416 - acc: 0.4671 - val_loss: 1.0154 - val_

10235/10235 [==============================] - 133s 13ms/step - loss: 1.0735 - acc: 0.4336 - val_loss: 1.0215 - val_acc: 0.5000
Epoch 2/5
10235/10235 [==============================] - 124s 12ms/step - loss: 1.0370 - acc: 0.4797 - val_loss: 1.0095 - val_acc: 0.5031
Epoch 3/5
10235/10235 [==============================] - 124s 12ms/step - loss: 1.0269 - acc: 0.4952 - val_loss: 1.0007 - val_acc: 0.5148
Epoch 4/5
10235/10235 [==============================] - 125s 12ms/step - loss: 1.0114 - acc: 0.5042 - val_loss: 1.0095 - val_acc: 0.5132
Epoch 5/5
1265/1265 [==============================] - 6s 5ms/step
0.507509881752753
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 138s 13ms/step - loss: 1.0652 - acc: 0.4345 - val_loss: 1.0294 - val_acc: 0.5055
Epoch 2/5
10235/10235 [==============================] - 125s 12ms/step - loss: 1.0373 - acc: 0.4734 - val_loss: 1.0120 - val_acc: 0.5093
Epoch 3/5
10235/10235 [=========================

10235/10235 [==============================] - 113s 11ms/step - loss: 1.0179 - acc: 0.5020 - val_loss: 1.0074 - val_acc: 0.5101
Epoch 5/5
1265/1265 [==============================] - 5s 4ms/step
0.5027667984896498
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 118s 12ms/step - loss: 1.0642 - acc: 0.4396 - val_loss: 1.0196 - val_acc: 0.4938
Epoch 2/5
10235/10235 [==============================] - 113s 11ms/step - loss: 1.0419 - acc: 0.4727 - val_loss: 1.0234 - val_acc: 0.5078
Epoch 3/5
10235/10235 [==============================] - 115s 11ms/step - loss: 1.0288 - acc: 0.4856 - val_loss: 1.0077 - val_acc: 0.5055
Epoch 4/5
10235/10235 [==============================] - 118s 11ms/step - loss: 1.0198 - acc: 0.4969 - val_loss: 1.0109 - val_acc: 0.5202
Epoch 5/5
1265/1265 [==============================] - 5s 4ms/step
0.485375494188942
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [============================

10235/10235 [==============================] - 124s 12ms/step - loss: 1.0096 - acc: 0.5057 - val_loss: 0.9964 - val_acc: 0.5187
Epoch 5/5
1265/1265 [==============================] - 6s 4ms/step
0.4980237156977295
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 139s 14ms/step - loss: 1.0707 - acc: 0.4367 - val_loss: 1.0412 - val_acc: 0.4548
Epoch 2/5
10235/10235 [==============================] - 120s 12ms/step - loss: 1.0401 - acc: 0.4781 - val_loss: 1.0096 - val_acc: 0.5101
Epoch 3/5
10235/10235 [==============================] - 122s 12ms/step - loss: 1.0221 - acc: 0.4986 - val_loss: 1.0191 - val_acc: 0.5047
Epoch 4/5
10235/10235 [==============================] - 123s 12ms/step - loss: 1.0128 - acc: 0.5006 - val_loss: 0.9964 - val_acc: 0.5179
Epoch 5/5
1265/1265 [==============================] - 7s 6ms/step
0.4814229252310138
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [===========================

10235/10235 [==============================] - 121s 12ms/step - loss: 1.0657 - acc: 0.4360 - val_loss: 1.0219 - val_acc: 0.5070
Epoch 2/5
10235/10235 [==============================] - 120s 12ms/step - loss: 1.0412 - acc: 0.4734 - val_loss: 1.0091 - val_acc: 0.5125
Epoch 3/5
10235/10235 [==============================] - 119s 12ms/step - loss: 1.0287 - acc: 0.4863 - val_loss: 1.0158 - val_acc: 0.5093
Epoch 4/5
10235/10235 [==============================] - 118s 12ms/step - loss: 1.0151 - acc: 0.5009 - val_loss: 0.9974 - val_acc: 0.5164
Epoch 5/5
1265/1265 [==============================] - 5s 4ms/step
0.509881423207611
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 117s 11ms/step - loss: 1.0634 - acc: 0.4439 - val_loss: 1.0297 - val_acc: 0.4868
Epoch 2/5
10235/10235 [==============================] - 115s 11ms/step - loss: 1.0407 - acc: 0.4763 - val_loss: 1.0132 - val_acc: 0.5117
Epoch 3/5
10235/10235 [=========================

10235/10235 [==============================] - 141s 14ms/step - loss: 1.0711 - acc: 0.4366 - val_loss: 1.0196 - val_acc: 0.5117
Epoch 2/5
10235/10235 [==============================] - 125s 12ms/step - loss: 1.0364 - acc: 0.4766 - val_loss: 1.0049 - val_acc: 0.5109
Epoch 3/5
10235/10235 [==============================] - 125s 12ms/step - loss: 1.0213 - acc: 0.4958 - val_loss: 1.0057 - val_acc: 0.5187
Epoch 4/5
10235/10235 [==============================] - 125s 12ms/step - loss: 1.0111 - acc: 0.5043 - val_loss: 0.9927 - val_acc: 0.5218
Epoch 5/5
1265/1265 [==============================] - 7s 6ms/step
0.49723320191085574
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 147s 14ms/step - loss: 1.0712 - acc: 0.4286 - val_loss: 1.0225 - val_acc: 0.4969
Epoch 2/5
10235/10235 [==============================] - 125s 12ms/step - loss: 1.0429 - acc: 0.4731 - val_loss: 1.0137 - val_acc: 0.4992
Epoch 3/5
10235/10235 [=======================

10235/10235 [==============================] - 115s 11ms/step - loss: 1.0167 - acc: 0.4962 - val_loss: 1.0020 - val_acc: 0.5109
Epoch 5/5
1265/1265 [==============================] - 5s 4ms/step
0.507509881446484
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 123s 12ms/step - loss: 1.0602 - acc: 0.4458 - val_loss: 1.0198 - val_acc: 0.5125
Epoch 2/5
10235/10235 [==============================] - 119s 12ms/step - loss: 1.0374 - acc: 0.4778 - val_loss: 1.0328 - val_acc: 0.4844
Epoch 3/5
10235/10235 [==============================] - 120s 12ms/step - loss: 1.0268 - acc: 0.4897 - val_loss: 1.0074 - val_acc: 0.5023
Epoch 4/5
10235/10235 [==============================] - 121s 12ms/step - loss: 1.0150 - acc: 0.4960 - val_loss: 1.0049 - val_acc: 0.5070
Epoch 5/5
1265/1265 [==============================] - 5s 4ms/step
0.5035573122765236
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [============================

10235/10235 [==============================] - 126s 12ms/step - loss: 1.0120 - acc: 0.5033 - val_loss: 0.9987 - val_acc: 0.5179
Epoch 5/5
1265/1265 [==============================] - 7s 6ms/step
0.5106719370416031
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 152s 15ms/step - loss: 1.0685 - acc: 0.4330 - val_loss: 1.0268 - val_acc: 0.4821
Epoch 2/5
10235/10235 [==============================] - 127s 12ms/step - loss: 1.0380 - acc: 0.4801 - val_loss: 1.0182 - val_acc: 0.5164
Epoch 3/5
10235/10235 [==============================] - 126s 12ms/step - loss: 1.0225 - acc: 0.4940 - val_loss: 1.0065 - val_acc: 0.5062
Epoch 4/5
10235/10235 [==============================] - 126s 12ms/step - loss: 1.0065 - acc: 0.5085 - val_loss: 1.0135 - val_acc: 0.5109
Epoch 5/5
1265/1265 [==============================] - 7s 6ms/step
0.5059288540847687
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [===========================

10235/10235 [==============================] - 301s 29ms/step - loss: 1.0638 - acc: 0.4336 - val_loss: 1.0194 - val_acc: 0.4907
Epoch 2/5
10235/10235 [==============================] - 287s 28ms/step - loss: 1.0384 - acc: 0.4781 - val_loss: 1.0085 - val_acc: 0.5132
Epoch 3/5
10235/10235 [==============================] - 224s 22ms/step - loss: 1.0247 - acc: 0.4932 - val_loss: 1.0013 - val_acc: 0.5109
Epoch 4/5
10235/10235 [==============================] - 142s 14ms/step - loss: 1.0123 - acc: 0.5024 - val_loss: 1.0020 - val_acc: 0.5171
Epoch 5/5
1265/1265 [==============================] - 5s 4ms/step
0.4972332016517051
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 160s 16ms/step - loss: 1.0653 - acc: 0.4365 - val_loss: 1.0252 - val_acc: 0.5187
Epoch 2/5
10235/10235 [==============================] - 138s 13ms/step - loss: 1.0384 - acc: 0.4785 - val_loss: 1.0072 - val_acc: 0.5179
Epoch 3/5
10235/10235 [========================

10235/10235 [==============================] - 163s 16ms/step - loss: 1.0770 - acc: 0.4317 - val_loss: 1.0303 - val_acc: 0.4961
Epoch 2/5
10235/10235 [==============================] - 141s 14ms/step - loss: 1.0416 - acc: 0.4709 - val_loss: 1.0174 - val_acc: 0.4945
Epoch 3/5
10235/10235 [==============================] - 137s 13ms/step - loss: 1.0239 - acc: 0.4954 - val_loss: 1.0038 - val_acc: 0.5078
Epoch 4/5
10235/10235 [==============================] - 127s 12ms/step - loss: 1.0123 - acc: 0.5000 - val_loss: 1.0056 - val_acc: 0.5179
Epoch 5/5
1265/1265 [==============================] - 7s 6ms/step
0.5122529648038239
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 167s 16ms/step - loss: 1.0735 - acc: 0.4315 - val_loss: 1.0252 - val_acc: 0.5093
Epoch 2/5
10235/10235 [==============================] - 140s 14ms/step - loss: 1.0392 - acc: 0.4754 - val_loss: 1.0253 - val_acc: 0.5023
Epoch 3/5
10235/10235 [========================

In [20]:
cnn_bert_rounds

[{5: 0.5067193679658791,
  6: 0.5162055339737843,
  7: 0.5130434782844288,
  8: 0.5225296443865705,
  9: 0.5098814232547293,
  10: 0.505928854131887,
  11: 0.5027667987016821,
  12: 0.507509881752753,
  13: 0.49644268807686365,
  14: 0.48458498056698224,
  15: 0.507509881752753,
  16: 0.4996047431536814,
  17: 0.5114624508755952,
  18: 0.49802371574484783,
  19: 0.5011857710808162,
  20: 0.5043478261105157,
  21: 0.5209486169777369,
  22: 0.516205533926666,
  23: 0.49723320191085574,
  24: 0.4758893284401875,
  25: 0.5098814233018476,
  26: 0.5043478264639029,
  27: 0.5035573126299108,
  28: 0.4877470355966817,
  29: 0.5114624509698318,
  30: 0.5083003953275944,
  31: 0.5209486169306186,
  32: 0.5090909094678555,
  33: 0.5035573125827925,
  34: 0.5169960478077764,
  35: 0.5035573125827925},
 {5: 0.5122529644975549,
  6: 0.5027667984896498,
  7: 0.485375494188942,
  8: 0.5177865616417685,
  9: 0.5106719371358397,
  10: 0.5138339924246897,
  11: 0.48695652180980786,
  12: 0.4861660080229

In [21]:
traces = cnn_bert_rounds

# Create traces
def create_scatter(counter):
    acc_dict = traces[counter]
    
    return go.Scatter(
        x = list(acc_dict.keys()),
        y = list(acc_dict.values()),
        mode = 'lines+markers',
        name = 'Round ' + str(counter)
    )

trace_data = [create_scatter(trace) for trace in range(len(traces))]

layout = go.Layout(
    title = 'CNN test set accuracy of padded BERT dataset with variable maximum lengths',
)

fig = go.Figure(data = trace_data, layout = layout)

iplot(fig)

In [7]:
concatenated_elmo = {
    dataset: [np.concatenate(np.array(statement)) for statement in elmo[dataset].statement]
    for dataset in elmo.keys()
}

In [8]:
%%time

cnn_elmo_rounds = [calculate_round(concatenated_elmo) for round in range(5)]

Instructions for updating:
Colocations handled automatically by placer.
2019-06-06 17:27:55,650 From /Users/martijn/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
2019-06-06 17:27:55,736 From /Users/martijn/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
2019-06-06 17:27:55,831 From /Users/martijn/anaconda3/lib/python3.6/site-packages/tensorflo

1265/1265 [==============================] - ETA:  - 5s 4ms/step
0.5185770751223734
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 126s 12ms/step - loss: 1.0726 - acc: 0.4361 - val_loss: 1.0170 - val_acc: 0.5008
Epoch 2/5
10235/10235 [==============================] - 122s 12ms/step - loss: 1.0353 - acc: 0.4823 - val_loss: 1.0163 - val_acc: 0.5078
Epoch 3/5
10235/10235 [==============================] - 122s 12ms/step - loss: 1.0192 - acc: 0.4979 - val_loss: 1.0048 - val_acc: 0.5156
Epoch 4/5
10235/10235 [==============================] - 122s 12ms/step - loss: 0.9997 - acc: 0.5165 - val_loss: 1.0008 - val_acc: 0.5140
Epoch 5/5
1265/1265 [==============================] - 5s 4ms/step
0.507509881752753
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 129s 13ms/step - loss: 1.0737 - acc: 0.4322 - val_loss: 1.0191 - val_acc: 0.5008
Epoch 2/5
10235/10235 [====================

1265/1265 [==============================] - 9s 7ms/step
0.513833992471808
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 167s 16ms/step - loss: 1.0716 - acc: 0.4357 - val_loss: 1.0388 - val_acc: 0.4899
Epoch 2/5
10235/10235 [==============================] - 139s 14ms/step - loss: 1.0345 - acc: 0.4812 - val_loss: 1.0053 - val_acc: 0.5101
Epoch 3/5
10235/10235 [==============================] - 140s 14ms/step - loss: 1.0126 - acc: 0.5059 - val_loss: 1.0064 - val_acc: 0.5109
Epoch 4/5
10235/10235 [==============================] - 139s 14ms/step - loss: 0.9965 - acc: 0.5195 - val_loss: 0.9943 - val_acc: 0.5234
Epoch 5/5
1265/1265 [==============================] - 8s 7ms/step
0.5003952569405552
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 173s 17ms/step - loss: 1.0696 - acc: 0.4360 - val_loss: 1.0258 - val_acc: 0.5000
Epoch 2/5
10235/10235 [============================

1265/1265 [==============================] - 5s 4ms/step
0.5130434782844288
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 127s 12ms/step - loss: 1.0715 - acc: 0.4344 - val_loss: 1.0310 - val_acc: 0.4938
Epoch 2/5
10235/10235 [==============================] - 122s 12ms/step - loss: 1.0360 - acc: 0.4796 - val_loss: 1.0304 - val_acc: 0.5078
Epoch 3/5
10235/10235 [==============================] - 123s 12ms/step - loss: 1.0128 - acc: 0.4983 - val_loss: 1.0062 - val_acc: 0.5156
Epoch 4/5
10235/10235 [==============================] - 123s 12ms/step - loss: 1.0000 - acc: 0.5127 - val_loss: 1.0049 - val_acc: 0.5234
Epoch 5/5
1265/1265 [==============================] - 5s 4ms/step
0.5098814233018476
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 128s 13ms/step - loss: 1.0702 - acc: 0.4283 - val_loss: 1.0165 - val_acc: 0.5008
Epoch 2/5
10235/10235 [===========================

1265/1265 [==============================] - 9s 7ms/step
0.5217391307646106
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 164s 16ms/step - loss: 1.0778 - acc: 0.4259 - val_loss: 1.0365 - val_acc: 0.4977
Epoch 2/5
10235/10235 [==============================] - 132s 13ms/step - loss: 1.0353 - acc: 0.4819 - val_loss: 1.0078 - val_acc: 0.5335
Epoch 3/5
10235/10235 [==============================] - 132s 13ms/step - loss: 1.0154 - acc: 0.5017 - val_loss: 0.9910 - val_acc: 0.5249
Epoch 4/5
10235/10235 [==============================] - 133s 13ms/step - loss: 0.9936 - acc: 0.5215 - val_loss: 0.9856 - val_acc: 0.5304
Epoch 5/5
1265/1265 [==============================] - 8s 6ms/step
0.5083003955867451
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 169s 16ms/step - loss: 1.0694 - acc: 0.4379 - val_loss: 1.0315 - val_acc: 0.4961
Epoch 2/5
10235/10235 [===========================

...

10235/10235 [==============================] - 163s 16ms/step - loss: 1.0771 - acc: 0.4252 - val_loss: 1.0210 - val_acc: 0.5031
Epoch 2/5
10235/10235 [==============================] - 131s 13ms/step - loss: 1.0373 - acc: 0.4782 - val_loss: 1.0115 - val_acc: 0.5187
Epoch 3/5
10235/10235 [==============================] - 129s 13ms/step - loss: 1.0168 - acc: 0.5003 - val_loss: 0.9971 - val_acc: 0.5312
Epoch 4/5
10235/10235 [==============================] - 129s 13ms/step - loss: 1.0000 - acc: 0.5188 - val_loss: 0.9941 - val_acc: 0.5241
Epoch 5/5
1265/1265 [==============================] - 8s 6ms/step
0.5114624509227135
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 162s 16ms/step - loss: 1.0850 - acc: 0.4295 - val_loss: 1.0199 - val_acc: 0.5070
Epoch 2/5
10235/10235 [==============================] - 129s 13ms/step - loss: 1.0409 - acc: 0.4747 - val_loss: 1.0107 - val_acc: 0.5055
Epoch 3/5
10235/10235 [========================

Epoch 2/5
10235/10235 [==============================] - 120s 12ms/step - loss: 1.0411 - acc: 0.4689 - val_loss: 1.0262 - val_acc: 0.5000
Epoch 3/5
10235/10235 [==============================] - 120s 12ms/step - loss: 1.0268 - acc: 0.4912 - val_loss: 1.0182 - val_acc: 0.5117
Epoch 4/5
10235/10235 [==============================] - 120s 12ms/step - loss: 1.0152 - acc: 0.5000 - val_loss: 1.0038 - val_acc: 0.5125
Epoch 5/5
1265/1265 [==============================] - 5s 4ms/step
0.5098814232547293
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 125s 12ms/step - loss: 1.0691 - acc: 0.4301 - val_loss: 1.0168 - val_acc: 0.5008
Epoch 2/5
10235/10235 [==============================] - 120s 12ms/step - loss: 1.0389 - acc: 0.4739 - val_loss: 1.0149 - val_acc: 0.5078
Epoch 3/5
10235/10235 [==============================] - 144s 14ms/step - loss: 1.0221 - acc: 0.4914 - val_loss: 0.9999 - val_acc: 0.5171
Epoch 4/5
10235/10235 [==============

10235/10235 [==============================] - 131s 13ms/step - loss: 1.0389 - acc: 0.4823 - val_loss: 1.0070 - val_acc: 0.5156
Epoch 3/5
10235/10235 [==============================] - 127s 12ms/step - loss: 1.0159 - acc: 0.5061 - val_loss: 0.9986 - val_acc: 0.5132
Epoch 4/5
10235/10235 [==============================] - 127s 12ms/step - loss: 0.9977 - acc: 0.5173 - val_loss: 0.9886 - val_acc: 0.5202
Epoch 5/5
1265/1265 [==============================] - 8s 7ms/step
0.5083003956338633
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 163s 16ms/step - loss: 1.0747 - acc: 0.4308 - val_loss: 1.0229 - val_acc: 0.4938
Epoch 2/5
10235/10235 [==============================] - 128s 13ms/step - loss: 1.0381 - acc: 0.4708 - val_loss: 1.0035 - val_acc: 0.5234
Epoch 3/5
10235/10235 [==============================] - 128s 13ms/step - loss: 1.0169 - acc: 0.5021 - val_loss: 0.9934 - val_acc: 0.5195
Epoch 4/5
10235/10235 [========================

Epoch 3/5
10235/10235 [==============================] - 124s 12ms/step - loss: 1.0300 - acc: 0.4841 - val_loss: 1.0144 - val_acc: 0.5171
Epoch 4/5
10235/10235 [==============================] - 126s 12ms/step - loss: 1.0172 - acc: 0.5008 - val_loss: 1.0105 - val_acc: 0.5125
Epoch 5/5
1265/1265 [==============================] - 6s 5ms/step
0.507509881446484
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 124s 12ms/step - loss: 1.0650 - acc: 0.4336 - val_loss: 1.0224 - val_acc: 0.4969
Epoch 2/5
10235/10235 [==============================] - 118s 12ms/step - loss: 1.0398 - acc: 0.4695 - val_loss: 1.0067 - val_acc: 0.5055
Epoch 3/5
10235/10235 [==============================] - 118s 12ms/step - loss: 1.0227 - acc: 0.4938 - val_loss: 1.0032 - val_acc: 0.5117
Epoch 4/5
10235/10235 [==============================] - 118s 11ms/step - loss: 1.0130 - acc: 0.5009 - val_loss: 0.9992 - val_acc: 0.5125
Epoch 5/5
1265/1265 [=================

10235/10235 [==============================] - 130s 13ms/step - loss: 1.0178 - acc: 0.5014 - val_loss: 0.9965 - val_acc: 0.5249
Epoch 4/5
10235/10235 [==============================] - 130s 13ms/step - loss: 1.0000 - acc: 0.5150 - val_loss: 0.9935 - val_acc: 0.5202
Epoch 5/5
1265/1265 [==============================] - 8s 7ms/step
0.5138339924246897
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 165s 16ms/step - loss: 1.0795 - acc: 0.4297 - val_loss: 1.0287 - val_acc: 0.4922
Epoch 2/5
10235/10235 [==============================] - 130s 13ms/step - loss: 1.0383 - acc: 0.4809 - val_loss: 1.0086 - val_acc: 0.5156
Epoch 3/5
10235/10235 [==============================] - 125s 12ms/step - loss: 1.0171 - acc: 0.5005 - val_loss: 0.9995 - val_acc: 0.5156
Epoch 4/5
10235/10235 [==============================] - 126s 12ms/step - loss: 0.9985 - acc: 0.5206 - val_loss: 0.9990 - val_acc: 0.5148
Epoch 5/5
1265/1265 [==========================

In [9]:
cnn_elmo_rounds

[{5: 0.49565217424287156,
  6: 0.5114624506164446,
  7: 0.5169960477606581,
  8: 0.5122529644504367,
  9: 0.49249011860063424,
  10: 0.49644268807686365,
  11: 0.5075098817998712,
  12: 0.5185770751223734,
  13: 0.507509881752753,
  14: 0.4988142295788399,
  15: 0.5043478264639029,
  16: 0.5122529647567056,
  17: 0.5067193679187609,
  18: 0.5090909094207372,
  19: 0.513833992471808,
  20: 0.5059288540847687,
  21: 0.5209486166243497,
  22: 0.49565217424287156,
  23: 0.5122529647567056,
  24: 0.5098814229484603,
  25: 0.5272727276025554,
  26: 0.5225296445514844,
  27: 0.5146245063058001,
  28: 0.5051383402507766,
  29: 0.5059288537784998,
  30: 0.5169960477606581,
  31: 0.5051383402507766,
  32: 0.513833992471808,
  33: 0.5003952569405552,
  34: 0.5106719370887214,
  35: 0.5019762849148083},
 {5: 0.5043478261105157,
  6: 0.5114624509698318,
  7: 0.5304347829385238,
  8: 0.5067193679658791,
  9: 0.499604743412832,
  10: 0.5130434782844288,
  11: 0.5098814233018476,
  12: 0.5051383402978

In [10]:
traces = cnn_elmo_rounds

# Create traces
def create_scatter(counter):
    acc_dict = traces[counter]
    
    return go.Scatter(
        x = list(acc_dict.keys()),
        y = list(acc_dict.values()),
        mode = 'lines+markers',
        name = 'Round ' + str(counter)
    )

trace_data = [create_scatter(trace) for trace in range(len(traces))]

layout = go.Layout(
    title = 'CNN test set accuracy of padded ELMo dataset with variable maximum lengths',
)

fig = go.Figure(data = trace_data, layout = layout)

iplot(fig)

In [23]:
x = list(range(5, 36))
x_rev = x[::-1]

# BERT
bert_matrix = np.transpose(np.array([np.array(list(acc_round.values())) for acc_round in cnn_bert_rounds]))
bert_y = [np.average(row) for row in bert_matrix]
bert_y_upper = [row.max() for row in bert_matrix]
bert_y_lower = [row.min() for row in bert_matrix]
bert_y_lower = bert_y_lower[::-1]

bert1 = go.Scatter(
    x = x + x_rev,
    y = bert_y_upper + bert_y_lower,
    fill = 'tozerox',
    fillcolor = 'rgba(0,100,80,0.2)',
    line = dict(color = 'rgba(255,255,255,0)'),
    showlegend = False,
    name = 'BERT',
)
bert2 = go.Scatter(
    x = x,
    y = bert_y,
    line = dict(color='rgb(0,100,80)'),
    mode = 'lines+markers',
    name = 'BERT',
)

# ELMo
elmo_matrix = np.transpose(np.array([np.array(list(acc_round.values())) for acc_round in cnn_elmo_rounds]))
elmo_y = [np.average(row) for row in elmo_matrix]
elmo_y_upper = [row.max() for row in elmo_matrix]
elmo_y_lower = [row.min() for row in elmo_matrix]
elmo_y_lower = elmo_y_lower[::-1]

elmo1 = go.Scatter(
    x = x + x_rev,
    y = elmo_y_upper + elmo_y_lower,
    fill = 'tozerox',
    fillcolor = 'rgba(0,176,246,0.2)',
    line = dict(color = 'rgba(255,255,255,0)'),
    showlegend = False,
    name = 'ELMo',
)
elmo2 = go.Scatter(
    x = x,
    y = elmo_y,
    line = dict(color='rgb(0,176,246)'),
    mode = 'lines+markers',
    name = 'ELMo',
)

data = [bert1, bert2, elmo1, elmo2]
layout = go.Layout(
    title = 'CNN test set accuracy of padded datasets with variable maximum lengths',
    paper_bgcolor = 'rgb(255,255,255)',
    plot_bgcolor = 'rgb(229,229,229)',
    xaxis = dict(
        gridcolor = 'rgb(255,255,255)',
        range = [5,35],
        showgrid = True,
        showline = False,
        showticklabels = True,
        tickcolor = 'rgb(127,127,127)',
        ticks = 'outside',
        zeroline = False
    ),
    yaxis=dict(
        gridcolor='rgb(255,255,255)',
        showgrid = True,
        showline = False,
        showticklabels = True,
        tickcolor = 'rgb(127,127,127)',
        ticks = 'outside',
        zeroline = False
    ),
)

fig = go.Figure(data = data, layout = layout)

iplot(fig)

<hr>

### References

```
@article{DBLP:journals/corr/Wang17j,
  author    = {William Yang Wang},
  title     = {"Liar, Liar Pants on Fire": {A} New Benchmark Dataset for Fake News
               Detection},
  journal   = {CoRR},
  volume    = {abs/1705.00648},
  year      = {2017},
  url       = {http://arxiv.org/abs/1705.00648},
  archivePrefix = {arXiv},
  eprint    = {1705.00648},
  timestamp = {Mon, 13 Aug 2018 16:48:58 +0200},
  biburl    = {https://dblp.org/rec/bib/journals/corr/Wang17j},
  bibsource = {dblp computer science bibliography, https://dblp.org}
}
```